# Functions

In [1]:
def compute_matchup_stats_s_r(match):
    s_itt = match["p1 tourney serve adv mod"]
    r_itt = match["p1 tourney return adv mod"]
    
    s_jtt = match["p2 tourney serve adv mod"]
    r_jtt = match["p2 tourney return adv mod"]
    
    # Pr[i wins serve on j at tournament t on day t]
    s_ijtt = (s_itt * (1 - r_jtt)) / ((s_itt * (1 - r_jtt)) + (r_jtt * (1 - s_itt)))
    # Pr[j wins serve on i at tournament t on day t]
    s_jitt = (s_jtt * (1 - r_itt)) / ((s_jtt * (1 - r_itt)) + (r_itt * (1 - s_jtt)))
    
    return s_ijtt, s_jitt

In [2]:
def compute_matchup_stats_sg_rg_s_r(match):
    s_itt = match["p1 tourney serve adv mod"]
    r_itt = match["p1 tourney return adv mod"]
    
    s_jtt = match["p2 tourney serve adv mod"]
    r_jtt = match["p2 tourney return adv mod"]
    
    sg_itt = match["p1 tourney service game adv mod"]
    rg_itt = match["p1 tourney return game adv mod"]
    
    sg_jtt = match["p2 tourney service game adv mod"]
    rg_jtt = match["p2 tourney return game adv mod"]
    
    # Pr[i wins serve on j at tournament t on day t]
    s_ijtt = (s_itt * (1 - r_jtt)) / ((s_itt * (1 - r_jtt)) + (r_jtt * (1 - s_itt)))
    # Pr[j wins serve on i at tournament t on day t]
    s_jitt = (s_jtt * (1 - r_itt)) / ((s_jtt * (1 - r_itt)) + (r_itt * (1 - s_jtt)))
    
    # Pr[i wins service game on j at tournament t on day t]
    sg_ijtt = (sg_itt * (1 - rg_jtt)) / ((sg_itt * (1 - rg_jtt)) + (rg_jtt * (1 - sg_itt)))
    # Pr[j wins service game on i at tournament t on day t]
    sg_jitt = (sg_jtt * (1 - rg_itt)) / ((sg_jtt * (1 - rg_itt)) + (rg_itt * (1 - sg_jtt)))
    
    
    return s_ijtt, s_jitt, sg_ijtt, sg_jitt

In [3]:
def compute_matchup_stats_sg_rg_t(match):
    sg_itt = match["p1 tourney service game adv mod"]
    rg_itt = match["p1 tourney return game adv mod"]
    
    sg_jtt = match["p2 tourney service game adv mod"]
    rg_jtt = match["p2 tourney return game adv mod"]
    
    t_itt = match["p1 tourney tiebreak adv mod"]
    t_jtt = match["p2 tourney tiebreak adv mod"]
    
    # Pr[i wins service game on j at tournament t on day t]
    sg_ijtt = (sg_itt * (1 - rg_jtt)) / ((sg_itt * (1 - rg_jtt)) + (rg_jtt * (1 - sg_itt)))
    # Pr[j wins service game on i at tournament t on day t]
    sg_jitt = (sg_jtt * (1 - rg_itt)) / ((sg_jtt * (1 - rg_itt)) + (rg_itt * (1 - sg_jtt)))
    
    # Pr[i wins tiebreak on j at tournament t on day t]
    t_ijtt = (t_itt * (1 - t_jtt)) / ((t_itt * (1 - t_jtt)) + (t_jtt * (1 - t_itt)))
    
    return sg_ijtt, sg_jitt, t_ijtt 

In [4]:
def compute_matchup_stats_set(match):
    sets_itt = match["p1 tourney set adv mod"]
    sets_jtt = match["p2 tourney set adv mod"]
    
    # Pr[i wins set on j at tournament t on day t]
    sets_ijtt = (sets_itt * (1 - sets_jtt)) / ((sets_itt * (1 - sets_jtt)) + (sets_jtt * (1 - sets_itt)))
    
    return sets_ijtt

In [5]:
def compute_log_loss_s_r(lambdas, *matches):
    matches = matches[0]
    sigma = lambdas[0]
    tao = lambdas[1]

    beta_serve_p1 = (sigma / matches["p1 year serve total"]) / (sigma / matches["p1 year serve total"] + tao)
    beta_serve_p2 = (sigma / matches["p2 year serve total"]) / (sigma / matches["p2 year serve total"] + tao)

    beta_return_p1 = (sigma / matches["p1 year return total"]) / (sigma / matches["p1 year return total"] + tao)
    beta_return_p2 = (sigma / matches["p2 year return total"]) / (sigma / matches["p2 year return total"] + tao)

    # compute shrunken win rates
    matches["p1 serve win rate mod"] = matches["p1 serve win rate"] * (1 - (beta_serve_p1)) + matches["all player year serve win rate"]* (beta_serve_p1)
    matches["p1 return win rate mod"] = matches["p1 return win rate"] * (1 - beta_return_p1) + (1- matches["all player year serve win rate"]) * (beta_return_p1)
    
    matches["p2 serve win rate mod"] = matches["p2 serve win rate"] * (1 - beta_serve_p2) + matches["all player year serve win rate"] * beta_serve_p2
    matches["p2 return win rate mod"] = matches["p2 return win rate"] * (1 - beta_return_p2) + (1- matches["all player year serve win rate"]) * beta_return_p2
    
    # compute opponent/ tournament adjusted serve win rate
    matches["p1 relative serve mod"] = matches["p1 serve win rate mod"] - (1 - matches["p1 opp return win rate"])
    matches["p1 relative return mod"] = matches["p1 return win rate mod"] - (1 - matches["p1 opp serve win rate"])
    
    matches["p2 relative return mod"] = matches["p2 return win rate mod"] - (1 - matches["p2 opp serve win rate"])
    matches["p2 relative serve mod"] = matches["p2 serve win rate mod"] - (1 - matches["p2 opp return win rate"])
    
    matches["p1 tourney serve adv mod"] = (matches["tourney serve win rate"] + matches["p1 relative serve mod"]).clip(upper=0.9985, lower=0.0015)
    matches["p1 tourney return adv mod"] = (matches["tourney return win rate"] + matches["p1 relative return mod"]).clip(upper=0.9985, lower=0.0015)
    
    matches["p2 tourney serve adv mod"] = (matches["tourney serve win rate"] + matches["p2 relative serve mod"]).clip(upper=0.9985, lower=0.0015)
    matches["p2 tourney return adv mod"] = (matches["tourney return win rate"] + matches["p2 relative return mod"]).clip(upper=0.9985, lower=0.0015)
    
    # Computing matchup statistics
    matchup_cols = ["p1 serve_tt", "p2 serve_tt"]
    matches[matchup_cols] = matches.apply(lambda match: compute_matchup_stats_s_r(match), axis=1, result_type="expand")

    games_after_2012 = matches[matches["date"] > datetime.strptime("01 Aug 12", '%d %b %y')]
    
    # Only consider matches on or after August 1, 2012, because we've had one year of data at that point
    games_after_2012 = matches[matches["date"] > datetime.strptime("01 Aug 12", '%d %b %y')]

    # Only consider best of three set matches
    games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

    # Don't consider Davis Cup matches because they don't have a final set tiebreak
    games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

    # Remove rows where either player hasn't played at least 10 matches in the past year
    games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]
    
    # Remove rows where we don't have s_ijtt values to predict the result of the match
    games_after_2012 = games_after_2012[(games_after_2012['p1 serve_tt'].notna()) & (games_after_2012['p2 serve_tt'].notna())]
    
    # compute Pr[p1 wins] with opponent/ tournament adjusted shrunken serve values
    games_after_2012[["p1 pr win"]] = games_after_2012.apply(lambda match: match_driver(2, 6, 4, 7, 1, match["p1 serve_tt"], match["p2 serve_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)
    
    games_after_2012[["winner"]] = games_after_2012["winner"].replace(2,0)
    
    compute_lloss = games_after_2012[["winner", "p1 pr win"]].copy()
    compute_lloss["lloss opp adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win"]) + (1-m["winner"])*np.log(1 - m["p1 pr win"])), axis=1)
    N = compute_lloss.shape[0]
    lloss_opp_adjust = (compute_lloss["lloss opp adjust"].sum()) * (-1/N)
    global Nfeval
    print(Nfeval)
    Nfeval += 1
    return lloss_opp_adjust


In [6]:
def compute_log_loss_sg_rg_s_r(lambdas, *matches):
    matches = matches[0]
    sigma1 = lambdas[0]
    tao1 = lambdas[1]
    sigma2 = lambdas[2]
    tao2 = lambdas[3]

    beta_sg_p1 = (sigma1 / matches["p1 year service game total"]) / (sigma1 / matches["p1 year service game total"] + tao1)
    beta_sg_p2 = (sigma1 / matches["p2 year service game total"]) / (sigma1 / matches["p2 year service game total"] + tao1)

    beta_rg_p1 = (sigma1 / matches["p1 year return game total"]) / (sigma1 / matches["p1 year return game total"] + tao1)
    beta_rg_p2 = (sigma1 / matches["p2 year return game total"]) / (sigma1 / matches["p2 year return game total"] + tao1)

    beta_serve_p1 = (sigma2 / matches["p1 year serve total"]) / (sigma2 / matches["p1 year serve total"] + tao2)
    beta_serve_p2 = (sigma2 / matches["p2 year serve total"]) / (sigma2 / matches["p2 year serve total"] + tao2)

    beta_return_p1 = (sigma2 / matches["p1 year return total"]) / (sigma2 / matches["p1 year return total"] + tao2)
    beta_return_p2 = (sigma2 / matches["p2 year return total"]) / (sigma2 / matches["p2 year return total"] + tao2)
    
    # compute shrunken win rates
    matches["p1 serve win rate mod"] = matches["p1 serve win rate"] * (1 - (beta_serve_p1)) + matches["all player year serve win rate"]* (beta_serve_p1)
    matches["p1 return win rate mod"] = matches["p1 return win rate"] * (1 - beta_return_p1) + (1- matches["all player year serve win rate"]) * (beta_return_p1)
    
    matches["p2 serve win rate mod"] = matches["p2 serve win rate"] * (1 - beta_serve_p2) + matches["all player year serve win rate"] * beta_serve_p2
    matches["p2 return win rate mod"] = matches["p2 return win rate"] * (1 - beta_return_p2) + (1- matches["all player year serve win rate"]) * beta_return_p2
    
    matches["p1 service game win rate mod"] = matches["p1 service game win rate"] * (1 - (beta_sg_p1)) + matches["all player year service game win rate"]* (beta_sg_p1)
    matches["p1 return game win rate mod"] = matches["p1 return game win rate"] * (1 - beta_rg_p1) + (1- matches["all player year service game win rate"]) * (beta_rg_p1)
    
    matches["p2 service game win rate mod"] = matches["p2 service game win rate"] * (1 - beta_sg_p2) + matches["all player year service game win rate"] * beta_sg_p2
    matches["p2 return game win rate mod"] = matches["p2 return game win rate"] * (1 - beta_rg_p2) + (1- matches["all player year service game win rate"]) * beta_rg_p2
    
    # compute opponent/ tournament adjusted serve win rate
    matches["p1 relative serve mod"] = matches["p1 serve win rate mod"] - (1 - matches["p1 opp return win rate"])
    matches["p1 relative return mod"] = matches["p1 return win rate mod"] - (1 - matches["p1 opp serve win rate"])
    
    matches["p2 relative return mod"] = matches["p2 return win rate mod"] - (1 - matches["p2 opp serve win rate"])
    matches["p2 relative serve mod"] = matches["p2 serve win rate mod"] - (1 - matches["p2 opp return win rate"])
    
    matches["p1 relative service game mod"] = matches["p1 service game win rate mod"] - (1 - matches["p1 opp return game win rate"])
    matches["p1 relative return game mod"] = matches["p1 return game win rate mod"] - (1 - matches["p1 opp service game win rate"])
    
    matches["p2 relative return game mod"] = matches["p2 return game win rate mod"] - (1 - matches["p2 opp service game win rate"])
    matches["p2 relative service game mod"] = matches["p2 service game win rate mod"] - (1 - matches["p2 opp return game win rate"])
    
    matches["p1 tourney serve adv mod"] = (matches["tourney serve win rate"] + matches["p1 relative serve mod"]).clip(upper=0.9985, lower=0.0015)
    matches["p1 tourney return adv mod"] = (matches["tourney return win rate"] + matches["p1 relative return mod"]).clip(upper=0.9985, lower=0.0015)
    
    matches["p2 tourney serve adv mod"] = (matches["tourney serve win rate"] + matches["p2 relative serve mod"]).clip(upper=0.9985, lower=0.0015)
    matches["p2 tourney return adv mod"] = (matches["tourney return win rate"] + matches["p2 relative return mod"]).clip(upper=0.9985, lower=0.0015)
    
    matches["p1 tourney service game adv mod"] = (matches["tourney service game win rate"] + matches["p1 relative service game mod"]).clip(upper=0.9985, lower=0.0015)
    matches["p1 tourney return game adv mod"] = (matches["tourney return game win rate"] + matches["p1 relative return game mod"]).clip(upper=0.9985, lower=0.0015)
    
    matches["p2 tourney service game adv mod"] = (matches["tourney service game win rate"] + matches["p2 relative service game mod"]).clip(upper=0.9985, lower=0.0015)
    matches["p2 tourney return game adv mod"] = (matches["tourney return game win rate"] + matches["p2 relative return game mod"]).clip(upper=0.9985, lower=0.0015)
    
    
    # Computing matchup statistics
    matchup_cols = ["p1 serve_tt", "p2 serve_tt", "p1 service_game_tt", "p2 service_game_tt"]
    matches[matchup_cols] = matches.apply(lambda match: compute_matchup_stats_sg_rg_s_r(match), axis=1, result_type="expand")

    games_after_2012 = matches[matches["date"] > datetime.strptime("01 Aug 12", '%d %b %y')]
    
    # Only consider matches on or after August 1, 2012, because we've had one year of data at that point
    games_after_2012 = matches[matches["date"] > datetime.strptime("01 Aug 12", '%d %b %y')]

    # Only consider best of three set matches
    games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

    # Don't consider Davis Cup matches because they don't have a final set tiebreak
    games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

    # Remove rows where either player hasn't played at least 10 matches in the past year
    games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]
    
    # Remove rows where we don't have s_ijtt values to predict the result of the match
    games_after_2012 = games_after_2012[(games_after_2012['p1 service_game_tt'].notna()) & (games_after_2012['p2 service_game_tt'].notna()) & (games_after_2012['p1 serve_tt'].notna()) & (games_after_2012['p2 serve_tt'].notna())]
    
    # compute Pr[p1 wins] with opponent/ tournament adjusted shrunken serve values
    games_after_2012["p1 pr win"] = games_after_2012.apply(lambda match: match_driver_game_based_tiebreak_serve_based(2, 6, 4, 7, 1, match["p1 service_game_tt"], match["p2 service_game_tt"], match["p1 serve_tt"], match["p2 serve_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)
    
    games_after_2012[["winner"]] = games_after_2012["winner"].replace(2,0)
    
    compute_lloss = games_after_2012[["winner", "p1 pr win"]].copy()
    compute_lloss["lloss opp adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win"]) + (1-m["winner"])*np.log(1 - m["p1 pr win"])), axis=1)
    N = compute_lloss.shape[0]
    lloss_opp_adjust = (compute_lloss["lloss opp adjust"].sum()) * (-1/N)
    global Nfeval
    print(Nfeval)
    Nfeval += 1
    return lloss_opp_adjust

In [7]:
def compute_log_loss_sg_rg_t(lambdas, *matches):
    matches = matches[0]
    sigma1 = lambdas[0]
    tao1 = lambdas[1]
    sigma2 = lambdas[2]
    tao2 = lambdas[3]

    beta_sg_p1 = (sigma1 / matches["p1 year service game total"]) / (sigma1 / matches["p1 year service game total"] + tao1)
    beta_sg_p2 = (sigma1 / matches["p2 year service game total"]) / (sigma1 / matches["p2 year service game total"] + tao1)

    beta_rg_p1 = (sigma1 / matches["p1 year return game total"]) / (sigma1 / matches["p1 year return game total"] + tao1)
    beta_rg_p2 = (sigma1 / matches["p2 year return game total"]) / (sigma1 / matches["p2 year return game total"] + tao1)

    beta_tiebreak_p1 = (sigma2 / matches["p1 year tiebreak total"]) / (sigma2 / matches["p1 year tiebreak total"] + tao2)
    beta_tiebreak_p2 = (sigma2 / matches["p2 year tiebreak total"]) / (sigma2 / matches["p2 year tiebreak total"] + tao2)
    
    # compute shrunken win rates
    matches["p1 tiebreak win rate mod"] = matches["p1 tiebreak win rate"] * (1 - (beta_tiebreak_p1)) + 0.5*(beta_tiebreak_p1)
    matches["p2 tiebreak win rate mod"] = matches["p2 tiebreak win rate"] * (1 - beta_tiebreak_p2) + (0.5)*(beta_tiebreak_p2)
    
    matches["p1 service game win rate mod"] = matches["p1 service game win rate"] * (1 - (beta_sg_p1)) + matches["all player year service game win rate"]* (beta_sg_p1)
    matches["p1 return game win rate mod"] = matches["p1 return game win rate"] * (1 - beta_rg_p1) + (1- matches["all player year service game win rate"]) * (beta_rg_p1)
    
    matches["p2 service game win rate mod"] = matches["p2 service game win rate"] * (1 - beta_sg_p2) + matches["all player year service game win rate"] * beta_sg_p2
    matches["p2 return game win rate mod"] = matches["p2 return game win rate"] * (1 - beta_rg_p2) + (1- matches["all player year service game win rate"]) * beta_rg_p2
    
    # compute opponent/ tournament adjusted serve win rate
    matches["p1 relative tiebreak mod"] = matches["p1 tiebreak win rate mod"] - (1 - matches["p1 opp tiebreak win rate"])
    matches["p2 relative tiebreak mod"] = matches["p2 tiebreak win rate mod"] - (1 - matches["p2 opp tiebreak win rate"])
    
    matches["p1 relative service game mod"] = matches["p1 service game win rate mod"] - (1 - matches["p1 opp return game win rate"])
    matches["p1 relative return game mod"] = matches["p1 return game win rate mod"] - (1 - matches["p1 opp service game win rate"])
    
    matches["p2 relative return game mod"] = matches["p2 return game win rate mod"] - (1 - matches["p2 opp service game win rate"])
    matches["p2 relative service game mod"] = matches["p2 service game win rate mod"] - (1 - matches["p2 opp return game win rate"])
    
    matches["p1 tourney tiebreak adv mod"] = (0.5 + matches["p1 relative tiebreak mod"]).clip(upper=0.9985, lower=0.0015)
    matches["p2 tourney tiebreak adv mod"] = (0.5 + matches["p2 relative tiebreak mod"]).clip(upper=0.9985, lower=0.0015)
    
    matches["p1 tourney service game adv mod"] = (matches["tourney service game win rate"] + matches["p1 relative service game mod"]).clip(upper=0.9985, lower=0.0015)
    matches["p1 tourney return game adv mod"] = (matches["tourney return game win rate"] + matches["p1 relative return game mod"]).clip(upper=0.9985, lower=0.0015)
    
    matches["p2 tourney service game adv mod"] = (matches["tourney service game win rate"] + matches["p2 relative service game mod"]).clip(upper=0.9985, lower=0.0015)
    matches["p2 tourney return game adv mod"] = (matches["tourney return game win rate"] + matches["p2 relative return game mod"]).clip(upper=0.9985, lower=0.0015)
    
    
    # Computing matchup statistics
    matchup_cols = ["p1 service_game_tt", "p2 service_game_tt", "p1 tiebreak_tt"]
    matches[matchup_cols] = matches.apply(lambda match: compute_matchup_stats_sg_rg_t(match), axis=1, result_type="expand")

    games_after_2012 = matches[matches["date"] > datetime.strptime("01 Aug 12", '%d %b %y')]
    
    # Only consider matches on or after August 1, 2012, because we've had one year of data at that point
    games_after_2012 = matches[matches["date"] > datetime.strptime("01 Aug 12", '%d %b %y')]

    # Only consider best of three set matches
    games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

    # Don't consider Davis Cup matches because they don't have a final set tiebreak
    games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

    # Remove rows where either player hasn't played at least 10 matches in the past year
    games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]
    
    # Remove rows where we don't have s_ijtt values to predict the result of the match
    games_after_2012 = games_after_2012[(games_after_2012['p1 service_game_tt'].notna()) & (games_after_2012['p2 service_game_tt'].notna()) & (games_after_2012['p1 tiebreak_tt'].notna())]
    
    # compute Pr[p1 wins] with opponent/ tournament adjusted shrunken serve values
    games_after_2012["p1 pr win"] = games_after_2012.apply(lambda match: match_driver_game_based(2, 6, 4, 7, 1, match["p1 service_game_tt"], match["p2 service_game_tt"], match["p1 tiebreak_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)
    
    games_after_2012[["winner"]] = games_after_2012["winner"].replace(2,0)
    
    compute_lloss = games_after_2012[["winner", "p1 pr win"]].copy()
    compute_lloss["lloss opp adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win"]) + (1-m["winner"])*np.log(1 - m["p1 pr win"])), axis=1)
    N = compute_lloss.shape[0]
    lloss_opp_adjust = (compute_lloss["lloss opp adjust"].sum()) * (-1/N)
    global Nfeval
    print(Nfeval)
    Nfeval += 1
    return lloss_opp_adjust

In [8]:
def compute_log_loss_set(lambdas, *matches):
    matches = matches[0]
    sigma1 = lambdas[0]
    tao1 = lambdas[1]

    beta_set_p1 = (sigma1 / matches["p1 year set total"]) / (sigma1 / matches["p1 year set total"] + tao1)
    beta_set_p2 = (sigma1 / matches["p2 year set total"]) / (sigma1 / matches["p2 year set total"] + tao1)
    
    # compute shrunken win rates
    matches["p1 set win rate mod"] = matches["p1 set win rate"] * (1 - (beta_set_p1)) + 0.5*(beta_set_p1)
    matches["p2 set win rate mod"] = matches["p2 set win rate"] * (1 - beta_set_p2) + (0.5)*(beta_set_p2)
    
    # compute opponent/ tournament adjusted serve win rate
    matches["p1 relative set mod"] = matches["p1 set win rate mod"] - (1 - matches["p1 opp set win rate"])
    matches["p2 relative set mod"] = matches["p2 set win rate mod"] - (1 - matches["p2 opp set win rate"])
    
    matches["p1 tourney set adv mod"] = (0.5 + matches["p1 relative set mod"]).clip(upper=0.9985, lower=0.0015)
    matches["p2 tourney set adv mod"] = (0.5 + matches["p2 relative set mod"]).clip(upper=0.9985, lower=0.0015)
    
    
    # Computing matchup statistics
    matchup_cols = ["p1 set_tt"]
    matches[matchup_cols] = matches.apply(lambda match: compute_matchup_stats_set(match), axis=1, result_type="expand")
    
    games_after_2012 = matches[matches["date"] > datetime.strptime("01 Aug 12", '%d %b %y')]
    
    # Only consider matches on or after August 1, 2012, because we've had one year of data at that point
    games_after_2012 = matches[matches["date"] > datetime.strptime("01 Aug 12", '%d %b %y')]

    # Only consider best of three set matches
    games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

    # Don't consider Davis Cup matches because they don't have a final set tiebreak
    games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

    # Remove rows where either player hasn't played at least 10 matches in the past year
    games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]
    
    # Remove rows where we don't have s_ijtt values to predict the result of the match
    games_after_2012 = games_after_2012[games_after_2012['p1 set_tt'].notna()]
    
    # compute Pr[p1 wins] with opponent/ tournament adjusted shrunken serve values
    games_after_2012["p1 pr win"] = games_after_2012.apply(lambda match: match_driver_set_based(2, match["p1 set_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)
    games_after_2012[["winner"]] = games_after_2012["winner"].replace(2,0)
    compute_lloss = games_after_2012[["winner", "p1 pr win"]].copy()
    compute_lloss["lloss opp adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win"]) + (1-m["winner"])*np.log(1 - m["p1 pr win"])), axis=1)
    N = compute_lloss.shape[0]
    lloss_opp_adjust = (compute_lloss["lloss opp adjust"].sum()) * (-1/N)
    global Nfeval
    print(Nfeval)
    Nfeval += 1
    return lloss_opp_adjust

### Try to predict games using $serve_{ijtt}$ values

In [16]:
from scipy.special import loggamma
from scipy.optimize import minimize, brute, fmin

Nfeval = 1

def callbackF(Xi):
    global Nfeval
    print('{0:4d}   {1: 3.6f} '.format(Nfeval, Xi[0]))
    Nfeval += 1

def calculate_optimal_lambda_s_r(matches):

    
    return brute(compute_log_loss_s_r, Ns=30, args=(matches,), ranges=((0.01,1000),(0.01,400),), finish=fmin)


In [243]:
brute_res = calculate_optimal_lambda_s_r(training_matches)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [244]:
brute_res

array([1106.56022809,   12.42877392])

In [245]:
compute_log_loss_s_r(brute_res, training_matches)

1026


0.630714760327189

In [ ]:
0.6307147608593233

In [64]:
np.divide(6, float("inf"))

0.0

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import beta
from multiprocessing import Pool, freeze_support
pd.set_option("display.max_columns", None)

In [10]:
# import the markov model functions lazily
%run Modeling_tennis_match_iid_points.ipynb

0.70147
0.933295


In [11]:
matches = pd.read_csv("./data/2011-2015_matches_cleaned_forShrinkage.csv")
matches["date"] = matches.apply(lambda x: datetime.strptime(x["date"], '%Y-%m-%d'), axis=1)
matches

,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes,p1 serves won,p1 serves total,p1 service games won,p1 service games total,p1 sets won,p1 tiebreaks won,p2 serves won,p2 serves total,p2 service games won,p2 service games total,p2 sets won,p2 tiebreaks won,sets total,tourney serves won,tourney serves total,tourney service games won,tourney service games total,p1 serve win rate,p1 return win rate,p1 service game win rate,p1 return game win rate,p1 set win rate,p1 tiebreak win rate,p2 serve win rate,p2 return win rate,p2 service game win rate,p2 return game win rate,p2 set win rate,p2 tiebreak win rate,p1 opp serve win rate,p1 opp return win rate,p1 opp service game win rate,p1 opp return game win rate,p1 opp set win rate,p1 opp tiebreak win rate,p2 opp serve win rate,p2 opp return win rate,p2 opp service game win rate,p2 opp return game win rate,p2 opp set win rate,p2 opp tiebreak win rate,p1 match count,p2 match count,all player year serve win rate,all player year service game win rate,p1 year serve total,p1 year return total,p1 year service game total,p1 year return game total,p1 year set total,p1 year tiebreak total,p2 year serve total,p2 year return total,p2 year service game total,p2 year return game total,p2 year set total,p2 year tiebreak total,p1 year match win rate,p2 year match rate,p1 year match results,p2 year match results,p1 opp match win rate,p2 opp match win rate,p1 relative serve,p2 relative serve,p1 relative return,p2 relative return,p1 relative service games,p2 relative service games,p1 relative return games,p2 relative return games,p1 relative set,p2 relative set,p1 relative tiebreak,p2 relative tiebreak,tourney serve win rate,tourney return win rate,tourney service game win rate,tourney return game win rate,p1 tourney serve adv,p1 tourney return adv,p2 tourney serve adv,p2 tourney return adv,p1 tourney service game adv,p1 tourney return game adv,p2 tourney service game adv,p2 tourney return game adv,p1 tourney set adv,p2 tourney set adv,p1 tourney tiebreak adv,p2 tourney tiebreak adv,tourney sets to win,p1 serve_tt,p1 service_game_tt,p1 set_tt,p1 tiebreak_tt,p2 serve_tt,p2 service_game_tt,p1 serve_tt no sos adjust,p2 serve_tt no sos adjust,p1 service_game_tt no sos adjust,p2 service_game_tt no sos adjust
0,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,2,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2231276,2011-07-28,ATPStu,ATP,Main,Robin Haase,Marin Cilic,2,SSRSS;RRSSRSSS;SSSS;RSSSS;SRSRSS;RSRSRSSS;RSRS...,4-6 6-4 6-3,0,141,60,102,11,15,1,0,60,92,12,14,2,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2231342,2011-07-28,Farmer,ATP,Main,Alejandro Falla,Thomaz Bellucci,2,SRRSSRSRRSRSRSRSRSRR;SSSRS;RRSRR;SRSSS;RRSSRR;...,6-0 6-1,0,42,21,55,1,7,0,0,26,37,6,6,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2229262,2011-07-28,Credit,ATP,Main,Matthias Bachinger,Julien Benneteau,2,SSSS;SSSS;RRRR;SSSS;SSSS;SRRRSSRSSRRSSRSS;SSSS...,6-4 6-4,0,57,36,59,7,10,0,0,43,58,9,10,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [12]:
# Calculate tournament averages
matches["tourney serve win rate"] = matches["tourney serves won"] / matches["tourney serves total"]
matches["tourney return win rate"] = 1 - matches["tourney serve win rate"]
matches["tourney service game win rate"] = matches["tourney service games won"] / matches["tourney service games total"]
matches["tourney return game win rate"] = 1 - matches["tourney service game win rate"]

In [13]:
# compute sets needed to win match (to use later in row filtering for evaluation)
# Computing sets needed to win the match
matches["tourney sets to win"] = matches[["p1 sets won", "p2 sets won"]].max(axis=1)

# change value of winner
matches["winner"].replace(2,0, inplace=True)
matches["p1 year serve total"].replace(0, float("nan"), inplace=True)
matches["p2 year serve total"].replace(0, float("nan"), inplace=True)
matches["p1 year return total"].replace(0, float("nan"), inplace=True)
matches["p2 year return total"].replace(0, float("nan"), inplace=True)
matches["p1 year service game total"].replace(0, float("nan"), inplace=True)
matches["p2 year service game total"].replace(0, float("nan"), inplace=True)
matches["p1 year return game total"].replace(0, float("nan"), inplace=True)
matches["p2 year return game total"].replace(0, float("nan"), inplace=True)
matches["p1 year set total"].replace(0, float("nan"), inplace=True)
matches["p2 year set total"].replace(0, float("nan"), inplace=True)
matches["p1 year tiebreak total"].replace(0, float("nan"), inplace=True)
matches["p2 year tiebreak total"].replace(0, float("nan"), inplace=True)

In [14]:
training_matches = matches[matches["date"] <= datetime.strptime("31 Dec 14", '%d %b %y')].copy()
test_matches = matches[(matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y'))&(matches["p1 match count"] >= 2) & (matches["p2 match count"] >= 2)&(matches["tourney sets to win"] == 2)].copy()                                                                                             
                                                                                             
                                                                                             

In [17]:
compute_log_loss_s_r(np.array([1106.56022809,   12.42877392]), test_matches)

1


0.6161055281212547

In [251]:
print(lambdas_s_r[0] / lambdas_s_r[1])
print(brute_res[0] / brute_res[1])


88.96745625045038
89.03213105682954


In [122]:
x_ = [0, 0.5, 100, 500, 1000, 9000]
for x in x_:
    print(x)
    print(compute_log_loss_s_r([x], training_matches))
    print("________________________")

0
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
8234    0.711887
8235    0.682680
8236    0.682311
8237    0.689249
8238    0.712271
Name: p1 serve win rate mod, Length: 8239, dtype: float64
0.6331015876738558
________________________
0.5
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
8234    0.711881
8235    0.682675
8236    0.682307
8237    0.689244
8238    0.712265
Name: p1 serve win rate mod, Length: 8239, dtype: float64
0.633058594799835
________________________
100
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
8234    0.710688
8235    0.681588
8236    0.681529
8237    0.688312
8238    0.711072
Name: p1 serve win rate mod, Length: 8239, dtype: float64
0.6324435909244187
________________________
500
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
8234    0.705893
8

### Try to predict using $sg_{ijtt}$ for games and $s_{ijtt}, s_{jitt}$ for tiebreaks

In [383]:
from scipy.special import loggamma
from scipy.optimize import minimize, brute, fmin

Nfeval = 1

def callbackF(Xi):
    global Nfeval
    print('{0:4d}   {1: 3.6f} '.format(Nfeval, Xi[0]))
    Nfeval += 1

def calculate_optimal_lambda_sg_rg_s_r(matches):

    lambdas = [1065.79028324,   26.23796037, 1065.79028324,   26.23796037]
    
    return minimize(compute_log_loss_sg_rg_s_r, x0=lambdas, args=(matches,), bounds=((0.01,None),(0.01,None),(0.01,None),(0.01,None),))



In [384]:
calculate_optimal_lambda_sg_rg_s_r(training_matches)

1
2
3
4
5
6
7
8
9
10


      fun: 0.6202820451654038
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.12133711e-06,  4.57411848e-05, -1.22125824e-07,  5.54001243e-06])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 10
      nit: 1
     njev: 2
   status: 0
  success: True
        x: array([1065.79028437,   26.23791465, 1065.7902834 ,   26.23795486])

In [18]:
lambdas = [1065.79028437,   26.23791465, 1065.7902834 ,   26.23795486]
compute_log_loss_sg_rg_s_r(lambdas, test_matches)

2


0.6039893837109304

In [388]:
import math
math.e**(-0.6049499547865307)

0.5461017557348002

In [301]:
math.log(math.e)

1.0

### Predict using $sg_{ijtt}$ and $rg_{ijtt}$ and $t_{ijtt}$

In [318]:
from scipy.special import loggamma
from scipy.optimize import minimize, brute, fmin

Nfeval = 1

def callbackF(Xi):
    global Nfeval
    print('{0:4d}   {1: 3.6f} '.format(Nfeval, Xi[0]))
    Nfeval += 1

def calculate_optimal_lambda_sg_rg_t(matches):

    lambdas = [1101.95310062,  507.52936855, 1106.39617461,   74.41308668]
    
    return minimize(compute_log_loss_sg_rg_t, x0=lambdas, args=(matches,), bounds=((0.01,None),(0.01,None),(0.01,None),(0.01,None),))



In [319]:
calculate_optimal_lambda_sg_rg_t(training_matches)

1
2
3
4
5


      fun: 0.6237306708251306
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.55432867e-07,  3.77475529e-07,  1.11023477e-07, -1.73194901e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 5
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([1101.95310062,  507.52936855, 1106.39617461,   74.41308668])

In [19]:
lambdas = [1101.95310062,  507.52936855, 1106.39617461,   74.41308668]
compute_log_loss_sg_rg_t(lambdas, test_matches)

3


0.6133226633472099

### Try to predict games using $sets_{ijtt}$ values

In [360]:
from scipy.special import loggamma
from scipy.optimize import minimize, brute, fmin

Nfeval = 1

def callbackF(Xi):
    global Nfeval
    print('{0:4d}   {1: 3.6f} '.format(Nfeval, Xi[0]))
    Nfeval += 1

def calculate_optimal_lambda_set(matches):

    lambdas = [1101.95310062,  509.52936855]
    
    return brute(compute_log_loss_set, Ns=30, args=(matches,), ranges=((0.01,1000),(0.01,800),), finish=fmin)



In [361]:
calculate_optimal_lambda_set(training_matches)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


array([1065.79028324,   26.23796037])

In [21]:
lambdas = [1065.79028324,   26.23796037]
compute_log_loss_set(lambdas, test_matches)

5


0.5971568297324278

In [25]:
import math
math.e**(-0.5971568297324278)

0.5503742213155763